In [1]:
import os, sys, json
import numpy as np

In [14]:
[2 ** i for i in range(5)]

[1, 2, 4, 8, 16]

In [15]:
epsilons = [0.1, 0.01, 0.001]
[0.0 for i in range(len(epsilons))]

[0.0, 0.0, 0.0]

In [16]:
10**0

1